In [1]:
import torch
from iTransformer import iTransformer

import scipy.io
import numpy as np

file_path = '/home/rithvik/iitm2/csi-prediction/EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)
data = data['EV_re_im_split']
print(data.shape)

data2=data[:,:-5,:]
data3=data[:,:5,:]

(50, 1000, 832)


In [2]:
data2.shape, data3.shape

((50, 995, 832), (50, 5, 832))

In [3]:
model = iTransformer(
    num_variates = 832,
    lookback_len = 995,                  # or the lookback length in the paper
    dim = 256,                          # model dimensions
    depth = 6,                          # depth
    heads = 8,                          # attention heads
    dim_head = 64,                      # head dimension
    pred_length = (5,),     # can be one prediction, or many
    num_tokens_per_variate = 1,         # experimental setting that projects each variate to more than one token. the idea is that the network can learn to divide up into time tokens for more granular attention across time. thanks to flash attention, you should be able to accommodate long sequence lengths just fine
    use_reversible_instance_norm = True # use reversible instance normalization, proposed here https://openreview.net/forum?id=cGDAkQo1C0p . may be redundant given the layernorms within iTransformer (and whatever else attention learns emergently on the first layer, prior to the first layernorm). if i come across some time, i'll gather up all the statistics across variates, project them, and condition the transformer a bit further. that makes more sense
)


Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


In [4]:
from torchinfo import summary
model_summary=summary(model,input_size=(50,995,832))
model_summary

Layer (type:depth-idx)                        Output Shape              Param #
iTransformer                                  [50, 5, 832]              1,024
├─RevIN: 1-1                                  [50, 832, 995]            (1,664)
├─Sequential: 1-2                             [50, 832, 256]            --
│    └─Linear: 2-1                            [50, 832, 256]            254,976
│    └─Rearrange: 2-2                         [50, 832, 256]            --
│    └─LayerNorm: 2-3                         [50, 832, 256]            512
├─ModuleList: 1-3                             --                        --
│    └─ModuleList: 2-4                        --                        --
│    │    └─Attention: 3-1                    [50, 836, 256]            655,360
│    │    └─LayerNorm: 3-2                    [50, 836, 256]            512
│    │    └─Sequential: 3-3                   [50, 836, 256]            525,396
│    │    └─LayerNorm: 3-4                    [50, 836, 256]          

In [5]:
# data2 = torch.from_numpy(data2).float()
values=torch.from_numpy(data2).float().to("cuda")
preds = model(values)
preds=preds[5]
preds.shape

OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 3.63 GiB of which 49.56 MiB is free. Including non-PyTorch memory, this process has 3.14 GiB memory in use. Of the allocated memory 2.96 GiB is allocated by PyTorch, and 121.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
type(data3),type(preds)

In [ ]:
# correct_predictions = (values == data3)
# accuracy = np.mean(correct_predictions)

# print(f'Element-wise comparison accuracy: {accuracy * 100:.2f}%')


In [ ]:
import numpy as np
import torch
from sklearn.metrics import mean_squared_error

# Assuming values is a PyTorch tensor and data3 is a NumPy array
# Detach the tensor and convert to numpy array
predictions_np = values.detach().numpy()
actuals_np = data3

# Reshape arrays to 2D if necessary
predictions_reshaped = predictions_np.reshape(-1, predictions_np.shape[-1])
actuals_reshaped = actuals_np.reshape(-1, actuals_np.shape[-1])

# Calculate Mean Squared Error
mse = mean_squared_error(actuals_reshaped, predictions_reshaped)
print(f'Mean Squared Error: {mse}')


In [ ]:
# # Assuming values is a PyTorch tensor and data3 is a NumPy array
# # Detach the tensor and convert to numpy array
# predictions_np = values.detach().numpy()
# actuals_np = data3

# # Calculate the 5% tolerance
# tolerance = 0.05 * actuals_np

# # Check if predictions are within 5% of actuals
# correct_predictions = np.abs(predictions_np - actuals_np) <= tolerance

# # Calculate accuracy
# accuracy = np.mean(correct_predictions)

# print(f'Accuracy within 5% error rate: {accuracy * 100:.2f}%')
